<a href="https://colab.research.google.com/github/mevius6/smallgan-exp/blob/master/smallgan_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Установка
[Здесь](https://github.com/apple2373/pytorch-small-dataset-image-generation) используется BigGAN для 128x128

[Авторский репоз для 256x256](https://github.com/nogu-atsu/small-dataset-image-generation)


---


[setup_dataloader_smallgan.py ](https://github.com/mevius6/ml-smallgan/blob/master/dataloaders/setup_dataloader_smallgan.py) — торч подгружает/трансформирует данные

[setup_model.py](https://github.com/mevius6/ml-smallgan/blob/master/models/setup_model.py) — конфиг для биггана

[BigGAN.py](https://github.com/mevius6/ml-smallgan/blob/master/models/BigGAN.py) — модель PyTorch (torch.nn.Module)  + *реализован дискриминатор*

[AdaBIGGAN.py](https://github.com/mevius6/ml-smallgan/blob/master/models/AdaBIGGAN.py) — адаптивная модель, реализует [авторский генератор](https://github.com/nogu-atsu/SmallGAN/blob/2293700dce1e2cd97e25148543532814659516bd/gen_models/ada_generator.py#L242-L294)

In [ ]:
!git clone https://github.com/mevius6/ml-smallgan.git

Cloning into 'ml-smallgan'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (390/390), done.
remote: Compressing objects: 100% (351/351), done.
remote: Total 390 (delta 74), reused 344 (delta 33), pack-reused 0
Receiving objects: 100% (390/390), 86.51 MiB | 37.52 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [ ]:
cd ml-smallgan

/content/ml-smallgan


[Основная контрольная точка для BigGAN](https://drive.google.com/open?id=1nAle7FCVFZdix2--ks0r5JBkFnKw8ctW), обученного в ImageNet с разрешением 128x128, с использованием [BS256](https://github.com/ajbrock/BigGAN-PyTorch/blob/master/scripts/sample_BigGAN_bs256x8.sh) и 8 градиентных накоплений, взятых перед коллапсом, с начальным показателем TF 97,35 +/- 1,79

In [ ]:
!gdown https://drive.google.com/open?id=1IIT8RJe7x3ghMaP5EfViIVAve_S_6F_Y

/usr/local/lib/python2.7/dist-packages/gdown/parse_url.py:31: UserWarning: You specified Google Drive Link but it is not the correct link to download the file. Maybe you should try: https://drive.google.com/uc?id=1IIT8RJe7x3ghMaP5EfViIVAve_S_6F_Y
  .format(url='https://drive.google.com/uc?id={}'.format(file_id))
Downloading...
From: https://drive.google.com/open?id=1IIT8RJe7x3ghMaP5EfViIVAve_S_6F_Y
To: /content/ml-smallgan/open?id=1IIT8RJe7x3ghMaP5EfViIVAve_S_6F_Y
185kB [00:00, 753kB/s]


In [ ]:
!unzip BigGAN_ch96_bs256x8_138k.zip

unzip:  cannot find or open BigGAN_ch96_bs256x8_138k.zip, BigGAN_ch96_bs256x8_138k.zip.zip or BigGAN_ch96_bs256x8_138k.zip.ZIP.


[Нужные веса](https://github.com/ajbrock/BigGAN-PyTorch/issues/8#issuecomment-477358969/) для генератора G.pth или G_ema.pth 

In [ ]:
!mv 138k/G_ema.pth data/

mv: cannot stat '138k/G_ema.pth': No such file or directory


## Обучение

In [ ]:
!wget https://www.dropbox.com/s/ikl574rrmuefyyv/checkpoint_iter25000.pth.tar

--2019-10-06 19:39:06--  https://www.dropbox.com/s/ikl574rrmuefyyv/checkpoint_iter25000.pth.tar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ikl574rrmuefyyv/checkpoint_iter25000.pth.tar [following]
--2019-10-06 19:39:06--  https://www.dropbox.com/s/raw/ikl574rrmuefyyv/checkpoint_iter25000.pth.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbd6816eeb911b4866e2df0fc65.dl.dropboxusercontent.com/cd/0/inline/Ap70TMrNplBHccOtscEnoFS2RXeyQRNhg24u9YU0b_W45TkMS_3avVVGGKYiHHXf2QcyQc0IOyXKY5mPtpOx3Er7a2IL2fjo3Qjk0Oj--KGDhw/file# [following]
--2019-10-06 19:39:06--  https://ucbd6816eeb911b4866e2df0fc65.dl.dropboxusercontent.com/cd/0/inline/Ap70TMrNplBHccOtscEnoFS2RXeyQRNhg24u9YU0b_W45TkMS_3avVVGGKYiHHXf2QcyQc0IOyXKY5mPtpOx3Er

In [ ]:
!python train.py --dataset rave --gpu 0 --pretrained ./data/G_ema.pth --resume "/content/ml-smallgan/checkpoint_iter25000.pth.tar" --iters 100

gpu id: 0
using device: cuda:0
made the log directory ./experiments/train_dataset-rave_model-biggan128-ada_2019-10-06-19-39-18
cannot import gitpython; try pip install gitpython
number of images (dataset size):  25
model name: biggan128-ada
Adding attention layer in G at resolution 64
Traceback (most recent call last):
  File "train.py", line 185, in <module>
    main(args)
  File "train.py", line 95, in main
    model = setup_model(args.model,dataset_size=dataset_size,resume=args.resume,biggan_imagenet_pretrained_model_path=args.pretrained)
  File "/content/ml-smallgan/models/setup_model.py", line 118, in setup_model
    G.load_state_dict(torch.load(biggan_imagenet_pretrained_model_path,map_location=lambda storage, loc: storage))
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 381, in load
    f = open(f, 'rb')
FileNotFoundError: [Errno 2] No such file or directory: './data/G_ema.pth'


## Тестирование



In [ ]:
import glob
import os
import matplotlib
from PIL import Image
import numpy as np
import json

%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torchvision
from models.setup_model import setup_model
from dataloaders.setup_dataloader_smallgan import setup_dataloader


def reconstruct(model,out_path,indices):
    model.eval()
    device = next(model.parameters()).device
    dataset_size = model.embeddings.weight.size()[0]
    assert type(indices)==torch.Tensor
    indices = indices.to(device)        
    embeddings = model.embeddings(indices)
    batch_size = embeddings.size()[0]
    image_tensors = model(embeddings)
    with torch.no_grad():
        torchvision.utils.save_image(
            image_tensors,
            out_path,
            nrow=int(batch_size ** 0.5),
            normalize=True,
        )
        
#see https://github.com/nogu-atsu/SmallGAN/blob/2293700dce1e2cd97e25148543532814659516bd/gen_models/ada_generator.py#L37-L53
def interpolate(model,out_path,source,dist,trncate=0.4,num=1):
    model.eval()
    device = next(model.parameters()).device
    dataset_size = model.embeddings.weight.size()[0]
    indices = torch.tensor([source,dist],device=device)
    indices = indices.to(device) 
    embeddings = model.embeddings(indices)
    embeddings = embeddings[[0]] * torch.linspace(1, 0, num,device=device)[:, None] + embeddings[[1]]* torch.linspace(0, 1, num,device=device)[:, None]
    batch_size = embeddings.size()[0]
    image_tensors = model(embeddings)
    with torch.no_grad():
        torchvision.utils.save_image(
            image_tensors,
            out_path,
            nrow=batch_size,
            normalize=True,
        )

#from https://github.com/nogu-atsu/SmallGAN/blob/2293700dce1e2cd97e25148543532814659516bd/gen_models/ada_generator.py#L37-L53        
def random(model,out_path,tmp=0.4, n=9, truncate=False):
    from scipy.stats import truncnorm
    model.eval()
    device = next(model.parameters()).device
    dataset_size = model.embeddings.weight.size()[0]
    dim_z = model.embeddings.weight.size(1)
    if truncate:
        embeddings = truncnorm(-tmp, tmp).rvs(n * dim_z).astype("float32").reshape(n, dim_z)
    else:
        embeddings = np.random.normal(0, tmp, size=(n, dim_z)).astype("float32")
    embeddings = torch.tensor(embeddings,device=device)
    batch_size = embeddings.size()[0]
    image_tensors = model(embeddings)
    with torch.no_grad():
        torchvision.utils.save_image(
            image_tensors,
            out_path,
            nrow=int(batch_size ** 0.5),
            normalize=True,
        )

In [ ]:
dataloader = setup_dataloader("rave",batch_size=2)
dataset_size = len(dataloader.dataset)
exp_dir = "./experiments/train_dataset-rave_model-biggan128-ada_2019-09-03-20-51-16/"
print(json.load(open(exp_dir+"args.json")))
model = setup_model("biggan128-ada",dataset_size=25,resume=exp_dir+"checkpoint_iter25000.pth.tar")
model = model.cuda()

FileNotFoundError: ignored

In [ ]:
reconstruct(model,out_path="./samples/rave_reconstruct.jpg",indices= torch.arange(9))
interpolate(model,out_path="./samples/rave_interpolate.jpg",source=5,dist=10)
random(model,out_path="./samples/rave_random.jpg",tmp=0.8, n=9, truncate=True)

plt.figure(figsize=(10,10))
im = Image.open("./samples/rave_reconstruct.jpg")
plt.imshow(im)
plt.show()

plt.figure(figsize=(20,20))
im = Image.open("./samples/rave_interpolate.jpg")
plt.imshow(im)
plt.show()

plt.figure(figsize=(10,10))
im = Image.open("./samples/rave_random.jpg")
plt.imshow(im)
plt.show()

In [ ]:
random(model,out_path="./samples/rave_random89.jpg",tmp=0.85, n=56, truncate=True)

plt.figure(figsize=(10,10))
im = Image.open("./samples/rave_random89.jpg")
plt.imshow(im)
plt.show()

In [ ]:
uploaded = drive.CreateFile({'title': 'rave_random2.jpg'})
uploaded.SetContentFile('./samples/rave_random2.jpg')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
uploaded = drive.CreateFile({'title': 'checkpoint_iter25000.pth.tar'})
uploaded.SetContentFile('./experiments/train_dataset-rave_model-biggan128-ada_2019-06-01-07-22-28/checkpoint_iter25000.pth.tar')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [ ]:
uploaded = drive.CreateFile({'title': 'train-log.json'})
uploaded.SetContentFile('./experiments/train_dataset-rave_model-biggan128-ada_2019-06-01-07-22-28/train-log.json')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))